some example code from: https://py-enigma.readthedocs.io/en/latest/index.html

source copied from: https://bitbucket.org/bgneal/enigma



In [10]:
#from enigma.machine import EnigmaMachine

# setup machine according to specs from a daily key sheet:

machine = EnigmaMachine.from_key_sheet(
       rotors='II IV V',
       reflector='B',
       ring_settings=[1, 20, 11],
       plugboard_settings='AV BS CG DL FU HZ IN KM OW RX')

# set machine initial starting position
machine.set_display('WXC')

# decrypt the message key
msg_key = machine.process_text('KCH')

# decrypt the cipher text with the unencrypted message key
machine.set_display(msg_key)

ciphertext = 'NIBLFMYMLLUFWCASCSSNVHAZ'
plaintext = machine.process_text(ciphertext)

print(plaintext)

THEXRUSSIANSXAREXCOMINGX


In [1]:
# keyfile.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""Contains a function to read key settings from a file.

A key file is expected to be formatted as one line per day of the month. Each
line consists of a sequence of space separated columns as follows:

day number - the first column is the day number (1-31). The lines can be in
any order.

rotor list - the next 3 or 4 columns should be rotor names. 

ring settings - the next 3 or 4 columns should be ring settings. They can be
in either alphabetic (A-Z) or numeric (0-25) formats.

plugboard settings - the next 10 columns should be plugboard settings. They
can be in either alphabetic (AB CD EF ...) or numeric (1/2 3/4 ...) formats.

reflector - the last column must be the reflector name.

Comment lines have a # character in the first column. Blank lines are ignored.

Each line must either have exactly 18 or 20 columns to be valid.

"""
import datetime


class KeyFileError(Exception):
    pass


def get_daily_settings(fp, day=None):
    """Read and parse a key file for daily key settings. 

    fp - a file-like object 

    day - specifies the day number to look for in the file (1-31). If day is
    None, the day number from today is used.
    
    Returns a dictionary of keyword arguments for EnigmaMachine.from_key_sheet.

    """
    if day is None:
        day = datetime.date.today().day

    for n, line in enumerate(fp):
        line = line.strip()
        if line == '' or line[0] == '#':
            continue

        cols = line.split()
        if len(cols) not in [18, 20]:
            raise KeyFileError("invalid column count on line %d" % n)

        rotor_count = 3 if len(cols) == 18 else 4

        try:
            day_num = int(cols[0])
        except ValueError:
            raise KeyFileError("invalid day on line %d" % n)

        if day_num != day:
            continue

        settings = {}
        if rotor_count == 3:
            settings['rotors'] = cols[1:4]
            settings['ring_settings'] = ' '.join(cols[4:7])
        else:
            settings['rotors'] = cols[1:5]
            settings['ring_settings'] = ' '.join(cols[5:9])

        settings['plugboard_settings'] = ' '.join(cols[-11:-1])
        settings['reflector'] = cols[-1]
        return settings

    else:
        raise KeyFileError('no entry for day %d found' % day)

        return settings

In [9]:
# machine.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""This module contains the top-level EnigmaMachine class for the Enigma Machine
simulation.

"""
import string
'''
from .rotors.factory import create_rotor, create_reflector
from .plugboard import Plugboard
from .keyfile import get_daily_settings
'''

class EnigmaError(Exception):
    pass

# The Enigma keyboard consists of the 26 letters of the alphabet, uppercase
# only:
KEYBOARD_CHARS = string.ascii_uppercase
KEYBOARD_SET = set(KEYBOARD_CHARS)


class EnigmaMachine:
    """Top-level class for the Enigma Machine simulation."""

    def __init__(self, rotors, reflector, plugboard):
        """Configures the Enigma Machine. Parameters are as follows:

        rotors - a list containing 3 or 4 (for the Kriegsmarine M4 version)
        Rotor objects. The order of the list is important. The first rotor is
        the left-most rotor, and the last rotor is the right-most (from the
        operator's perspective sitting at the machine).

        reflector - a rotor object to represent the reflector (UKW)

        plugboard - a plugboard object to represent the state of the plugboard

        Note that on the military Enigma machines we are simulating, the entry
        wheel is a simple straight-pass through and is not simulated here. It
        would not be too hard to add a parameter for the entry wheel and pass a
        Rotor object for it if it is desired to simulate a non-military Enigma
        machine.

        """
        if len(rotors) not in [3, 4]:
            raise EnigmaError("Must supply 3 or 4 rotors")

        self.rotors = rotors
        self.rotor_count = len(rotors)
        self.reflector = reflector
        self.plugboard = plugboard

    @classmethod
    def from_key_sheet(cls, rotors='I II III', ring_settings=None,
            reflector='B', plugboard_settings=None):

        """Convenience function to build an EnigmaMachine from the data as you
        might find it on a key sheet:

        rotors: either a list of strings naming the rotors from left to right
        or a single string:
            e.g. ["I", "III", "IV"] or "I III IV"

        ring_settings: either a list/tuple of integers, a string, or None to
        represent the ring settings to be applied to the rotors in the rotors
        list. The acceptable values are:
            - A list/tuple of integers with values between 0-25
            - A string; either space separated letters or numbers, e.g. 'B U L'
              or '2 21 12'. If numbers are used, they are 1-based to match
              historical key sheet data.
            - None means all ring settings are 0.

        reflector: a string that names the reflector to use

        plugboard_settings: a string of plugboard settings as you might find
        on a key sheet; e.g. 
            'PO ML IU KJ NH YT GB VF RE DC' 
        or
            '18/26 17/4 21/6 3/16 19/14 22/7 8/1 12/25 5/9 10/15'

            A value of None means no plugboard connections are made.

        """
        # validate inputs
        if isinstance(rotors, str):
            rotors = rotors.split()

        num_rotors = len(rotors)
        if num_rotors not in (3, 4):
            raise EnigmaError("invalid rotors list size")

        if ring_settings is None:
            ring_settings = [0] * num_rotors
        elif isinstance(ring_settings, str):
            strings = ring_settings.split()
            ring_settings = []
            for s in strings:
                if s.isalpha():
                    ring_settings.append(ord(s.upper()) - ord('A'))
                elif s.isdigit():
                    ring_settings.append(int(s) - 1)
                else:
                    raise EnigmaError('invalid ring setting: %s' % s)

        if num_rotors != len(ring_settings):
            raise EnigmaError("# of rotors doesn't match # of ring settings")

        # assemble the machine
        rotor_list = [create_rotor(r[0], r[1]) for r in zip(rotors, ring_settings)]

        return cls(rotor_list, 
                   create_reflector(reflector),
                   Plugboard.from_key_sheet(plugboard_settings))

    @classmethod
    def from_key_file(cls, fp, day=None):
        """Convenience function to read key parameters from a file.

        fp - a file-like object that contains daily key settings
        day - the line labeled with the day number (1-31) will be used for the
        settings. If day is None, the day number will be determined from today's
        date. 

        For more information on the file format, see keyfile.py.

        """
        args = get_daily_settings(fp, day)
        return cls.from_key_sheet(**args)

    def set_display(self, val):
        """Sets the rotor operator windows to 'val'.

        'val' must be a string or iterable containing values for each window
        from left to right.

        """
        if len(val) != self.rotor_count:
            raise EnigmaError("Incorrect length for display value")

        for i, rotor in enumerate(reversed(self.rotors)):
            rotor.set_display(val[-1 - i])

    def get_display(self):
        """Returns the operator display as a string."""

        return "{}{}{}".format(self.rotors[-3].get_display(),
                               self.rotors[-2].get_display(),
                               self.rotors[-1].get_display())

    def key_press(self, key):
        """Simulate a front panel key press. 

        key - a string representing the letter pressed

        The rotors are stepped by simulating the mechanical action of the
        machine. 
        Next a simulated current is run through the machine.
        The lamp that is lit by this key press is returned as a string.

        """
        if key not in KEYBOARD_SET:
            raise EnigmaError('illegal key press %s' % key)

        # simulate the mechanical action of the machine
        self._step_rotors()

        # simulate the electrical operations:
        signal_num = ord(key) - ord('A')
        lamp_num = self._electric_signal(signal_num)
        return KEYBOARD_CHARS[lamp_num]

    def _step_rotors(self):
        """Simulate the mechanical action of pressing a key."""
        
        # The right-most rotor's right-side ratchet is always over a pawl, and
        # it has no neighbor to the right, so it always rotates.
        #
        # The middle rotor will rotate if either:
        #   1) The right-most rotor's left side notch is over the 2nd pawl
        #       or
        #   2) It has a left-side notch over the 3rd pawl
        #
        # The third rotor (from the right) will rotate only if the middle rotor
        # has a left-side notch over the 3rd pawl.
        #
        # Kriegsmarine model M4 has 4 rotors, but the 4th rotor (the leftmost)
        # does not rotate (they did not add a 4th pawl to the mechanism).

        rotor1 = self.rotors[-1]
        rotor2 = self.rotors[-2]
        rotor3 = self.rotors[-3]

        # decide which rotors can move
        rotate2 = rotor1.notch_over_pawl() or rotor2.notch_over_pawl()
        rotate3 = rotor2.notch_over_pawl()

        # move rotors
        rotor1.rotate()
        if rotate2:
            rotor2.rotate()
        if rotate3:
            rotor3.rotate()

    def _electric_signal(self, signal_num):
        """Simulate running an electric signal through the machine in order to
        perform an encrypt or decrypt operation

        signal_num - the wire (0-25) that the simulated current occurs on

        Returns a lamp number to light (an integer 0-25).

        """
        pos = self.plugboard.signal(signal_num)

        for rotor in reversed(self.rotors):
            pos = rotor.signal_in(pos)

        pos = self.reflector.signal_in(pos)

        for rotor in self.rotors:
            pos = rotor.signal_out(pos)

        return self.plugboard.signal(pos)

    def process_text(self, text, replace_char='X'):
        """Run the text through the machine, simulating a key press for each
        letter in the text.

        text - the text to process. Note that the text is converted to upper
        case before processing.

        replace_char - if text contains a character not on the keyboard, replace
        it with replace_char; if replace_char is None the character is dropped
        from the message

        """
        result = []
        for key in text:
            c = key.upper()

            if c not in KEYBOARD_SET: 
                if replace_char:
                    c = replace_char
                else:
                    continue    # ignore it

            result.append(self.key_press(c))

        return ''.join(result)

    def get_rotor_counts(self):
        """Return the rotor rotation counts as a list of integers."""
        return [r.rotations for r in self.rotors]

In [0]:
#  main.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""Provide an example command-line app that can setup an EnigmaMachine and
process text.

"""

import argparse
import sys

from .machine import EnigmaMachine, EnigmaError
from .rotors import RotorError
from .keyfile import KeyFileError


PROG_DESC = 'Encrypt/decrypt text according to Enigma machine key settings'

HELP_EPILOG = """\
Key settings can either be specified by command-line arguments, or read
from a key file. If reading from a key file, the line labeled with the
current day number is used unless the --day argument is provided.

Text to process can be supplied 3 ways:

   if --text=TEXT is present TEXT is processed
   if --file=FILE is present the contents of FILE are processed
   otherwise the text is read from standard input

Examples:

    $ %(prog)s --key-file=enigma.keys -s XYZ -t HELLOXWORLDX
    $ %(prog)s -r III IV V -i 1 2 3 -p AB CD EF GH IJ KL MN -u B -s XYZ
    $ %(prog)s -r Beta III IV V -i A B C D -p 1/2 3/4 5/6 -u B-Thin -s WXYZ
  
"""

def create_from_key_file(filename, day=None):
    """Create an EnigmaMachine from a daily key sheet."""

    with open(filename, 'r') as f:
        return EnigmaMachine.from_key_file(f, day)


def create_from_args(parser, args):
    """Create an EnigmaMachine from command-line specs."""

    if args.rotors is None:
        parser.error("Please specify 3 or 4 rotors; e.g. II IV V")
    elif len(args.rotors) not in [3, 4]:
        parser.error("Expecting 3 or 4 rotors; %d supplied" % len(args.rotors))

    if args.text and args.file:
        parser.error("Please specify --text or --file, but not both")

    ring_settings = ' '.join(args.ring_settings) if args.ring_settings else None
    plugboard = ' '.join(args.plugboard) if args.plugboard else None

    return EnigmaMachine.from_key_sheet(rotors=args.rotors,
                                        ring_settings=ring_settings,
                                        plugboard_settings=plugboard,
                                        reflector=args.reflector)


def main():

    parser = argparse.ArgumentParser(description=PROG_DESC, epilog=HELP_EPILOG,
            formatter_class=argparse.RawDescriptionHelpFormatter)
    
    parser.add_argument('-k', '--key-file',
            help='path to key file for daily settings')
    parser.add_argument('-d', '--day', type=int, default=None,
            help='use the settings for day DAY when reading key file')
    parser.add_argument('-r', '--rotors', nargs='+', metavar='ROTOR',
            help='rotor list ordered from left to right; e.g III IV I')
    parser.add_argument('-i', '--ring-settings', nargs='+',
            metavar='RING_SETTING',
            help='ring setting list from left to right; e.g. A A J')
    parser.add_argument('-p', '--plugboard', nargs='+', metavar='PLUGBOARD',
            help='plugboard settings')
    parser.add_argument('-u', '--reflector', help='reflector name')
    parser.add_argument('-s', '--start', help='starting position')
    parser.add_argument('-t', '--text', help='text to process')
    parser.add_argument('-f', '--file', help='input file to process')
    parser.add_argument('-x', '--replace-char', default='X',
            help=('if the input text contains chars not found on the enigma'
                  ' keyboard, replace with this char [default: %(default)s]'))
    parser.add_argument('-z', '--delete-chars', default=False,
            action='store_true',
            help=('if the input text contains chars not found on the enigma'
                  ' keyboard, delete them from the input'))
    parser.add_argument('-v', '--verbose', action='store_true', default=False,
            help='provide verbose output; include final rotor positions')

    args = parser.parse_args()

    if args.key_file and (args.rotors or args.ring_settings or args.plugboard
            or args.reflector):
        parser.error("Please specify either a key file or command-line key "
                     "settings, but not both")

    if args.start is None:
        parser.error("Please specify a start position")

    if args.key_file:
        machine = create_from_key_file(args.key_file, args.day)
    else:
        machine = create_from_args(parser, args)

    if args.text:
        text = args.text
    elif args.file:
        with open(args.file, 'r') as f:
            text = f.read()
    else:
        text = input('--> ')

    replace_char = args.replace_char if not args.delete_chars else None

    machine.set_display(args.start)

    s = machine.process_text(text, replace_char=replace_char)

    if args.verbose:
        print('Final rotor positions:', machine.get_display())
        print('Rotor rotation counts:', machine.get_rotor_counts())
        print('Output:')

    print(s)


def console_main():
    try:
        main()
    except (IOError, EnigmaError, RotorError, KeyFileError) as ex:
        sys.stderr.write("%s\n" % ex)


if __name__ == '__main__':
    console_main()

In [3]:
# plugboard.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""Contains the Plugboard class for simulating the plugboard (Steckerbrett)
component of the Enigma Machine.

"""

import collections
import copy
from itertools import chain
import string


# On Heer & Luftwaffe (?) models, the plugs are labeled with upper case letters
HEER_LABELS = string.ascii_uppercase

# The number of plugboard cables supplied with a machine:
MAX_PAIRS = 10


class PlugboardError(Exception):
    pass


class Plugboard:
    """The plugboard allows the operator to swap letters before and after the 
    entry wheel. This is accomplished by connecting cables between pairs of
    plugs that are marked with letters (Heer & Luftwaffe models) or numbers
    (Kriegsmarine). Ten cables were issued with each machine; thus up to 10 of
    these swappings could be used as part of a machine setup.

    Each cable swaps both the input and output signals. Thus if A is connected
    to B, A crosses to B in the keyboard to entry wheel direction and also in
    the reverse entry wheel to lamp direction.

    """
    def __init__(self, wiring_pairs=None):
        """Configure the plugboard according to a list or tuple of integer
        pairs, or None.

        A value of None or an empty list/tuple indicates no plugboard
        connections are to be used (i.e. a straight mapping).

        Otherwise wiring_pairs must be an iterable of integer pairs, where each
        integer is between 0-25, inclusive. At most 10 such pairs can be
        specified. Each value represents an input/output path through the
        plugboard. It is invalid to specify the same path more than once in the
        list.

        If an invalid wiring_pairs parameter is given, a PlugboardError is
        raised.

        """
        # construct wiring mapping table with default 1-1 mappings
        self.wiring_map = list(range(26))

        # construct backup storage for the wiring map; this is useful when
        # hill-climbing and is used when the Plugboard is used as a context
        # manager
        self._backup_map = list(range(26))

        # use settings if provided
        if not wiring_pairs:
            return

        if len(wiring_pairs) > MAX_PAIRS:
            raise PlugboardError('Please specify %d or less pairs' % MAX_PAIRS)

        # ensure a path occurs at most once in the list
        counter = collections.Counter(chain.from_iterable(wiring_pairs))
        path, count = counter.most_common(1)[0]
        if count != 1:
            raise PlugboardError('duplicate connection: %d' % path)

        # make the connections
        for pair in wiring_pairs:
            m = pair[0]
            n = pair[1]
            if not (0 <= m < 26) or not (0 <= n < 26):
                raise PlugboardError('invalid connection: %s' % str(pair))

            self.wiring_map[m] = n
            self.wiring_map[n] = m

    @classmethod
    def from_key_sheet(cls, settings=None):
        """Configure the plugboard according to a settings string as you may
        find on a key sheet.

        Two syntaxes are supported, the Heer/Luftwaffe and Kriegsmarine styles:

        In the Heer syntax, the settings are given as a string of
        alphabetic pairs. For example: 'PO ML IU KJ NH YT GB VF RE DC' 

        In the Kriegsmarine syntax, the settings are given as a string of number
        pairs, separated by a '/'. Note that the numbering uses 1-26, inclusive.
        For example: '18/26 17/4 21/6 3/16 19/14 22/7 8/1 12/25 5/9 10/15'

        To specify no plugboard connections, settings can be None or an empty
        string.

        A PlugboardError will be raised if the settings string is invalid, or if
        it contains more than MAX_PAIRS pairs. Each plug should be present at
        most once in the settings string.

        """
        if not settings:
            return cls(None)

        wiring_pairs = []
        
        # detect which syntax is being used
        if settings.find('/') != -1:
            # Kriegsmarine syntax
            pairs = settings.split()
            for p in pairs:
                try:
                    m, n = p.split('/')
                    m, n = int(m), int(n)
                except ValueError:
                    raise PlugboardError('invalid pair: %s' % p)

                wiring_pairs.append((m - 1, n - 1))
        else:
            # Heer/Luftwaffe syntax
            pairs = settings.upper().split()

            for p in pairs:
                if len(p) != 2:
                    raise PlugboardError('invalid pair: %s' % p)

                m = p[0]
                n = p[1]
                if m not in HEER_LABELS or n not in HEER_LABELS:
                    raise PlugboardError('invalid pair: %s' % p)

                wiring_pairs.append((ord(m) - ord('A'), ord(n) - ord('A')))

        return cls(wiring_pairs)

    def get_pairs(self):
        """Return the connections as a set of tuple pairs."""
        pairs = set()
        for x in range(0, 26):
            y = self.wiring_map[x]
            if x != y and (y, x) not in pairs:
                pairs.add((x, y))

        return pairs

    def army_str(self):
        """Return settings as a string as found on an army key sheet."""
        pairs = list(self.get_pairs())
        pairs.sort()
        return ' '.join('{}{}'.format(chr(t[0] + ord('A')), 
                                      chr(t[1] + ord('A'))) for t in pairs)

    def navy_str(self):
        """Return settings as a string as found on a navy key sheet."""
        pairs = list(self.get_pairs())
        pairs.sort()
        return ' '.join('{}/{}'.format(t[0] + 1, t[1] + 1) for t in pairs)

    def __str__(self):
        """Returns a string representation of the settings in army format."""
        return self.army_str()

    def signal(self, n):
        """Simulate a signal entering the plugboard on wire n, where n must be
        an integer between 0 and 25.

        Returns the wire number of the output signal (0-25).

        Note that since the plugboard always crosses pairs of wires, it doesn't
        matter what direction (keyboard -> entry wheel or vice versa) the signal
        is coming from.

        """
        return self.wiring_map[n]

    # Support for hill-climbing algorithms:

    def get_wiring(self):
        """Returns a deep copy of the internal wiring map."""
        return copy.deepcopy(self.wiring_map)

    def is_wired(self, n):
        """Returns True if connection n has a cable attached; 0 <= n < 26."""
        return self.wiring_map[n] != n

    def is_free(self, n):
        """Returns True if connection n has no cable attached; 0 <= n < 26."""
        return self.wiring_map[n] == n

    def __enter__(self):
        """Saves the current state of the wiring map."""
        for n in range(26):
            self._backup_map[n] = self.wiring_map[n]
        return self

    def __exit__(self, *exc_info):
        """Restores the saved state of the wiring map."""
        for n in range(26):
            self.wiring_map[n] = self._backup_map[n]

    def connection(self, n):
        """Returns plug number [0-25] for what is connected to plug n [0-25]."""
        return self.wiring_map[n]

    def disconnect(self, n):
        """Removes cable from plug number n [0-25]."""
        x = self.wiring_map[n]
        self.wiring_map[x] = x
        self.wiring_map[n] = n

    def connect(self, x, y):
        """Connects plug x to plug y, removing any existing connections first.
        
        x and y must be in [0-25].

        """
        # disconnect any existing connections
        m = self.wiring_map[x]
        n = self.wiring_map[y]
        self.wiring_map[m] = m
        self.wiring_map[n] = n

        self.wiring_map[x] = y
        self.wiring_map[y] = x

    def is_connected(self, x, y):
        """Returns True if x is connected to y.

        x and y must be in [0-25].

        """
        return self.wiring_map[x] == y and self.wiring_map[y] == x

In [4]:
#  data.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""data.py - This file contains rotor & reflector data for all the types
we simulate.

"""

# This data is taken from Dirk Rijmenants very informative and useful Enigma
# website: "Techical Details of the Engigma Machine"
# http://users.telenet.be/d.rijmenants/en/enigmatech.htm
#
# Rotors I-V were used by the Heer, Luftwaffe, and Kriegsmarine. The
# Kriegsmarine added rotors VI-VIII to the M3 model, and added Beta & Gamma to
# the M4 model (used with thin reflectors only). Note that Beta & Gamma rotors
# did not rotate.
#
# The Heer, Luftwaffe, & Kriegsmarine M3 machines used reflectors B & C,
# while the Kriegsmarine M4 used thin reflectors B & C.
#

ROTORS = {
    'I': {
        'wiring': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ',
        'stepping': 'Q',
    },
    'II': {
        'wiring': 'AJDKSIRUXBLHWTMCQGZNPYFVOE',
        'stepping': 'E',
    },
    'III': {
        'wiring': 'BDFHJLCPRTXVZNYEIWGAKMUSQO',
        'stepping': 'V',
    },
    'IV': {
        'wiring': 'ESOVPZJAYQUIRHXLNFTGKDCMWB',
        'stepping': 'J',
    },
    'V': {
        'wiring': 'VZBRGITYUPSDNHLXAWMJQOFECK',
        'stepping': 'Z',
    },
    'VI': {
        'wiring': 'JPGVOUMFYQBENHZRDKASXLICTW',
        'stepping': 'ZM',
    },
    'VII': {
        'wiring': 'NZJHGRCXMYSWBOUFAIVLPEKQDT',
        'stepping': 'ZM',
    },
    'VIII': {
        'wiring': 'FKQHTLXOCBJSPDZRAMEWNIUYGV',
        'stepping': 'ZM',
    },
    'Beta': {
        'wiring': 'LEYJVCNIXWPBQMDRTAKZGFUHOS',
        'stepping': None,
    },
    'Gamma': {
        'wiring': 'FSOKANUERHMBTIYCWLQPZXVGJD',
        'stepping': None,
    },
}

REFLECTORS = {
    'B': 'YRUHQSLDPXNGOKMIEBFZCWVJAT',
    'C': 'FVPJIAOYEDRZXWGCTKUQSBNMHL',
    'B-Thin': 'ENKQAUYWJICOPBLMDXZVFTHRGS',
    'C-Thin': 'RDOBJNTKVEHMLFCWZAXGYIPSUQ',
}

In [8]:
#  factory.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""Contains factory functions for creating rotors and reflectors."""
'''
from . import RotorError
from .rotor import Rotor
from .data import ROTORS, REFLECTORS
'''

def create_rotor(model, ring_setting=0):
    """Factory function to create and return a rotor of the given model name."""

    if model in ROTORS:
        data = ROTORS[model]
        return Rotor(model, data['wiring'], ring_setting, data['stepping'])

    raise RotorError("Unknown rotor type: %s" % model)


def create_reflector(model):
    """Factory function to create and return a reflector of the given model
    name.
    
    """
    if model in REFLECTORS:
        return Rotor(model, wiring=REFLECTORS[model])

    raise RotorError("Unknown reflector type: %s" % model)

In [7]:
# rotor.py
# Copyright (C) 2012 by Brian Neal.
# This file is part of Py-Enigma, the Enigma Machine simulation.
# Py-Enigma is released under the MIT License (see License.txt).

"""rotor.py - this module contains the Rotor class for the Enigma simulation."""

import string
import collections

# from import RotorError <- caused error


ALPHA_LABELS = string.ascii_uppercase

# In specifying a wiring for a rotor, every letter must be included exactly
# once. This variable helps us enforce that:
WIRING_FREQ_SET = set((letter, 1) for letter in ALPHA_LABELS)


class Rotor:
    """The Rotor class represents the Enigma Machine rotors (Walzen).
    
    A rotor has 26 circularly arranged pins on the right (entry) side and 26
    contacts on the left side. Each pin is connected to a single contact by
    internal wiring, thus establishing a substitution cipher. We represent this
    wiring by establishing a mapping from a pin to a contact (and vice versa for
    the return path). Internally we number the pins and contacts from 0-25 in a
    clockwise manner with 0 being the "top".

    An alphabetic or numeric ring is fastened to the rotor by the operator. The
    labels of this ring are displayed to the operator through a small window on
    the top panel. The ring can be fixed to the rotor in one of 26 different
    positions; this is called the ring setting (Ringstellung). We will number
    the ring settings from 0-25 where 0 means no offset (e.g. the letter "A" is
    mapped to pin 0 on an alphabetic ring). A ring setting of 1 means the letter
    "B" is mapped to pin 0.

    Each rotor can be in one of 26 positions on the spindle, with position 0
    where pin/contact 0 is being indicated in the operator window. The rotor
    rotates towards the operator by mechanical means during normal operation as
    keys are being pressed during data entry. Position 1 is thus defined to be
    one step from position 0. Likewise, position 25 is the last position before
    another step returns it to position 0, completing 1 trip around the spindle.

    Finally, a rotor has a "stepping" or "turnover" parameter. Physically this
    is implemented by putting a notch on the alphabet ring and it controls when
    the rotor will "kick" the rotor to its left, causing the neighbor rotor to
    rotate. Most rotors had one notch, but some Kriegsmarine rotors had 2
    notches and thus rotated twice as fast.

    Note that due to the system of ratchets and pawls, the middle rotor (in a 3
    rotor Enigma) can "double-step". The middle rotor will advance on the next
    step of the first rotor a second time in a row, if the middle rotor is in
    its own turnover position.

    Note that we allow the stepping parameter to be None. This indicates the
    rotor does not rotate. This allows us to model the entry wheel and
    reflectors as stationary rotors.
    
    """

    def __init__(self, model_name, wiring, ring_setting=0, stepping=None):
        """Establish rotor characteristics:

        model_name - e.g. "I", "II", "III", "Beta", "Gamma"

        wiring - this should be a string of 26 alphabetic characters that
        represents the internal wiring transformation of the signal as it enters
        from the right side. This is the format used in various online
        resources. For example, for the Wehrmacht Enigma type I rotor the
        mapping is "EKMFLGDQVZNTOWYHXUSPAIBRCJ".

        ring_setting - this should be an integer from 0-25, inclusive, which
        indicates the Ringstellung. A value of 0 means there is no offset; e.g.
        the letter "A" is fixed to pin 0. A value of 1 means "B" is mapped to
        pin 0.

        stepping - this is the stepping or turnover parameter. It should be an
        iterable, for example a string such as "Q". This will indicate that when
        the rotor transitions from "Q" to "R" (by observing the operator
        window), the rotor will "kick" the rotor to its left, causing it to
        rotate. If the rotor has more than one notch, a string of length 2 could
        be used, e.g. "ZM".  Another way to think of this parameter is that when
        a character in the stepping string is visible in the operator window, a
        notch is lined up with the pawl on the left side of the rotor.  This
        will allow the pawl to push up on the rotor *and* the rotor to the left
        when the next key is depressed.

        Note that for purposes of simulation, our rotors will always use
        alphabetic labels A-Z. In reality, the Heer & Luftwaffe devices used
        numbers 01-26, and Kriegsmarine devices used A-Z. Our usage of A-Z is
        simply for simulation convenience. In the future we may allow either
        display.

        """
        self.name = model_name
        self.wiring_str = wiring.upper()
        self.ring_setting = ring_setting
        self.pos = 0
        self.rotations = 0

        # check wiring length
        if len(self.wiring_str) != 26:
            raise RotorError("invalid wiring length")

        # check wiring format; must contain A-Z
        for c in self.wiring_str:
            if c not in ALPHA_LABELS:
                raise RotorError("invalid wiring: %s" % wiring)

        # check wiring format; ensure every letter appears exactly once
        input_set = set(collections.Counter(self.wiring_str).items())
        if input_set != WIRING_FREQ_SET:
            raise RotorError("invalid wiring frequency")

        if not isinstance(ring_setting, int) or not (0 <= ring_setting < 26):
            raise RotorError("invalid ring_setting")

        # Create two lists to describe the internal wiring. Two lists are used
        # to do fast lookup from both entry (from the right) and exit (from the
        # left). 
        self.entry_map = [ord(pin) - ord('A') for pin in self.wiring_str]
        
        self.exit_map = [0] * 26
        for i, v in enumerate(self.entry_map):
            self.exit_map[v] = i

        # build a map of display values to positions
        self.display_map = {}
        for n in range(26):
            self.display_map[chr(ord('A') + n)] = (n - self.ring_setting) % 26

        # build a reverse map of position mapped to display values
        self.pos_map = {v : k for k, v in self.display_map.items()}

        # build step set; this is a set of positions where our notches are in
        # place to allow the pawls to move
        self.step_set = set()
        if stepping is not None:
            for pos in stepping:
                if pos in self.display_map:
                    self.step_set.add(pos)
                else:
                    raise RotorError("stepping: %s" % pos)

        # initialize our position and display value:
        self.set_display('A')

    def set_display(self, val):
        """Spin the rotor such that the string val appears in the operator
        window.

        This sets the internal position of the rotor on the axle and thus
        rotates the pins and contacts accordingly.

        A value of 'A' for example puts the rotor in position 0, assuming an
        internal ring setting of 0.

        The parameter val must be a string in 'A' - 'Z'.

        Setting the display resets the internal rotation counter to 0.

        """
        s = val.upper()
        if s not in self.display_map:
            raise RotorError("bad display value %s" % val)

        self.pos = self.display_map[s]
        self.display_val = s
        self.rotations = 0

    def get_display(self):
        """Returns what is currently being displayed in the operator window."""
        return self.display_val

    def signal_in(self, n):
        """Simulate a signal entering the rotor from the right at a given pin
        position n.

        n must be an integer between 0 and 25.

        Returns the contact number of the output signal (0-25).

        """
        # determine what pin we have at that position due to rotation
        pin = (n + self.pos) % 26

        # run it through the internal wiring
        contact = self.entry_map[pin]

        # turn back into a position due to rotation
        return (contact - self.pos) % 26

    def signal_out(self, n):
        """Simulate a signal entering the rotor from the left at a given
        contact position n.

        n must be an integer between 0 and 25.

        Returns the pin number of the output signal (0-25).

        """
        # determine what contact we have at that position due to rotation
        contact = (n + self.pos) % 26

        # run it through the internal wiring
        pin = self.exit_map[contact]

        # turn back into a position due to rotation
        return (pin - self.pos) % 26

    def notch_over_pawl(self):
        """Return True if this rotor has a notch in the stepping position and
        False otherwise.

        """
        return self.display_val in self.step_set

    def rotate(self):
        """Rotate the rotor forward due to mechanical stepping action."""

        self.pos = (self.pos + 1) % 26
        self.display_val = self.pos_map[self.pos]
        self.rotations += 1